# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [1]:
import sys
!{sys.executable} -m pip install gmaps

In [2]:
!jupyter nbextension enable --py --sys-prefix widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [3]:
!jupyter nbextension enable --py --sys-prefix gmaps

Enabling notebook extension jupyter-gmaps/extension...
      - Validating: ok


In [4]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [5]:
# Set the file path to import the WeatherPy_database.csv file
file_path = r"C:\Users\Xthea\OneDrive\Documents\Admin\Bootcamp\Carleton University\Classes\Module 6\Weather_Database\WeatherPy_Database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Chapais,CA,49.7834,-74.8492,9.88,99,100,1.35,overcast clouds
1,1,Airai,TL,-8.9266,125.4092,23.05,63,82,0.97,broken clouds
2,2,Atuona,PF,-9.8000,-139.0333,24.91,69,40,9.31,scattered clouds
3,3,Busselton,AU,-33.6500,115.3333,14.21,69,58,7.83,broken clouds
4,4,Port Alfred,ZA,-33.5906,26.8910,15.10,90,0,0.45,clear sky
5,5,Punta Arenas,CL,-53.1500,-70.9167,7.06,76,75,3.60,broken clouds
6,6,Yellowknife,CA,62.4560,-114.3525,2.29,64,75,5.66,broken clouds
7,7,Bluff,NZ,-46.6000,168.3333,4.33,68,94,13.79,overcast clouds
8,8,Edremit,TR,39.5961,27.0244,13.86,72,0,4.12,clear sky
9,9,Rikitea,PF,-23.1203,-134.9692,21.44,61,77,10.79,broken clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [6]:
# Prompt the user to enter the minimum temperature criteria
min_temp = input("What is your minimum temperature criteria for your vacation?")
min_temp = pd.to_numeric(min_temp)

# Prompt the user to enter the maximum temperature criteria
max_temp = input("What is your maximum temperature criteria for your vacation?")
max_temp = pd.to_numeric(max_temp)

What is your minimum temperature criteria for your vacation?15
What is your maximum temperature criteria for your vacation?25


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [7]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
                                    
# Display sample data
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Airai,TL,-8.9266,125.4092,23.05,63,82,0.97,broken clouds
2,2,Atuona,PF,-9.8000,-139.0333,24.91,69,40,9.31,scattered clouds
4,4,Port Alfred,ZA,-33.5906,26.8910,15.10,90,0,0.45,clear sky
9,9,Rikitea,PF,-23.1203,-134.9692,21.44,61,77,10.79,broken clouds
11,11,Nantucket,US,41.2835,-70.0995,17.06,94,100,7.20,mist
...,...,...,...,...,...,...,...,...,...,...
702,702,Yafran,LY,32.0633,12.5286,16.67,78,1,2.64,clear sky
703,703,Eureka,US,40.8021,-124.1637,15.67,80,77,2.24,broken clouds
706,706,Northam,GB,51.0333,-4.2167,16.25,97,100,9.19,overcast clouds
708,708,Roura,GF,4.7263,-52.3274,23.95,94,20,0.00,few clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [8]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Airai,TL,-8.9266,125.4092,23.05,63,82,0.97,broken clouds
2,2,Atuona,PF,-9.8000,-139.0333,24.91,69,40,9.31,scattered clouds
4,4,Port Alfred,ZA,-33.5906,26.8910,15.10,90,0,0.45,clear sky
9,9,Rikitea,PF,-23.1203,-134.9692,21.44,61,77,10.79,broken clouds
11,11,Nantucket,US,41.2835,-70.0995,17.06,94,100,7.20,mist
...,...,...,...,...,...,...,...,...,...,...
702,702,Yafran,LY,32.0633,12.5286,16.67,78,1,2.64,clear sky
703,703,Eureka,US,40.8021,-124.1637,15.67,80,77,2.24,broken clouds
706,706,Northam,GB,51.0333,-4.2167,16.25,97,100,9.19,overcast clouds
708,708,Roura,GF,4.7263,-52.3274,23.95,94,20,0.00,few clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [9]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City", "Country", 
                               "Max Temp", "Current Description",
                              "Lat", "Lng"]].copy()

# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng
1,Airai,TL,23.05,broken clouds,-8.9266,125.4092
2,Atuona,PF,24.91,scattered clouds,-9.8000,-139.0333
4,Port Alfred,ZA,15.10,clear sky,-33.5906,26.8910
9,Rikitea,PF,21.44,broken clouds,-23.1203,-134.9692
11,Nantucket,US,17.06,mist,41.2835,-70.0995
...,...,...,...,...,...,...
702,Yafran,LY,16.67,clear sky,32.0633,12.5286
703,Eureka,US,15.67,broken clouds,40.8021,-124.1637
706,Northam,GB,16.25,overcast clouds,51.0333,-4.2167
708,Roura,GF,23.95,few clouds,4.7263,-52.3274


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [10]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df['Hotel Name'] = ' '

# Display sample data
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Airai,TL,23.05,broken clouds,-8.9266,125.4092,
2,Atuona,PF,24.91,scattered clouds,-9.8000,-139.0333,
4,Port Alfred,ZA,15.10,clear sky,-33.5906,26.8910,
9,Rikitea,PF,21.44,broken clouds,-23.1203,-134.9692,
11,Nantucket,US,17.06,mist,41.2835,-70.0995,
...,...,...,...,...,...,...,...
702,Yafran,LY,16.67,clear sky,32.0633,12.5286,
703,Eureka,US,15.67,broken clouds,40.8021,-124.1637,
706,Northam,GB,16.25,overcast clouds,51.0333,-4.2167,
708,Roura,GF,23.95,few clouds,4.7263,-52.3274,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [11]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [12]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
#    print(lat, lng)
#     print(row)
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    #print(hotels)
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [13]:
# Display sample data
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Airai,TL,23.05,broken clouds,-8.9266,125.4092,
2,Atuona,PF,24.91,scattered clouds,-9.8000,-139.0333,Villa Enata
4,Port Alfred,ZA,15.10,clear sky,-33.5906,26.8910,The Halyards Hotel
9,Rikitea,PF,21.44,broken clouds,-23.1203,-134.9692,People ThankYou
11,Nantucket,US,17.06,mist,41.2835,-70.0995,Jared Coffin House


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [21]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df

clean_hotel_df = clean_hotel_df[clean_hotel_df["Hotel Name"] != " "]
#clean_hotel_df = hotel_df.drop(hotel_df.index[hotel_df['Hotel Name'] == ""], inplace=False)

# Display sample data
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Atuona,PF,24.91,scattered clouds,-9.8000,-139.0333,Villa Enata
4,Port Alfred,ZA,15.10,clear sky,-33.5906,26.8910,The Halyards Hotel
9,Rikitea,PF,21.44,broken clouds,-23.1203,-134.9692,People ThankYou
11,Nantucket,US,17.06,mist,41.2835,-70.0995,Jared Coffin House
15,Bandarbeyla,SO,24.51,overcast clouds,9.4942,50.8122,JABIR HOTEL
...,...,...,...,...,...,...,...
702,Yafran,LY,16.67,clear sky,32.0633,12.5286,بيوت الشباب بيفرن
703,Eureka,US,15.67,broken clouds,40.8021,-124.1637,Eureka Inn Trademark By Wyndham
706,Northam,GB,16.25,overcast clouds,51.0333,-4.2167,Durrant House Hotel
708,Roura,GF,23.95,few clouds,4.7263,-52.3274,Amazonie Découverte SARL


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [15]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [22]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [23]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [24]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>Villa Enata</dd>\n<dt>City</dt><dd>Atuona</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 24.91 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>The Halyards Hotel</dd>\n<dt>City</dt><dd>Port Alfred</dd>\n<dt>Country</dt><dd>ZA</dd>\n<dt>Current Weather</dt><dd>clear sky and 15.1 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>People ThankYou</dd>\n<dt>City</dt><dd>Rikitea</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Weather</dt><dd>broken clouds and 21.44 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Jared Coffin House</dd>\n<dt>City</dt><dd>Nantucket</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Weather</dt><dd>mist and 17.06 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>JABIR HOTEL</dd>\n<dt>City</dt><dd>Bandarbeyla</dd>\n<dt>Country</dt><dd>SO</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 24.51 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Ravinala</dd>\n<dt>City</dt><dd>T

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [25]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(30)

,Lat,Lng
2,-9.8000,-139.0333
4,-33.5906,26.8910
9,-23.1203,-134.9692
11,41.2835,-70.0995
15,9.4942,50.8122
16,-18.7667,46.0333
17,-33.0153,27.9116
19,5.5577,95.3222
25,-21.2000,-175.2000
30,-3.6819,13.3499


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [20]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig = gmaps.figure()
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))